In [5]:
# Execute this cell to check if the python driver for mongodb is installed correctly or not
import pymongo

In [6]:
#  Creating a connection with Mongo Client

from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)

In [7]:
# Get the existing database object 
db = client['training-database']

# If the data base does not exist it creates one, but 
# Collections and databases are created only when the first document is inserted into them.
# The below comand will not show anything in the DB , untill a document is inserted in a collection of this DB
db_new = client['training-new']

In [24]:
#get list of existing collection

print("List of existing collection")
collection_list = db.collection_names()
print(collection_list)
print()

# If no collection existes returns an empty list
print("List of existing collection")
collection_list_new = db_new.collection_names()
print(collection_list_new)
print()

# Get the collection using name
collection = db['training-collection']

print(collection)

List of existing collection
['training-collection']

List of existing collection
['collection-new']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'training-database'), 'training-collection')


In [9]:
# Inserting a document in mongodb collection

import datetime
post = {"author": "Mike1",
         "text": "My first blog post!",
         "tags": ["mongodb", "python", "pymongo"],
         "date": datetime.datetime.utcnow()}

# existing collection
post_id = collection.insert_one(post).inserted_id
print(post_id)

#If the collection does not exist a new collection is created
collection_new = db_new['collection-new']
post_id_new = collection_new.insert_one({'post':'test'}).inserted_id
print(post_id_new)

5bbd9ff7053b0633fcd61bcb
5bbd9ff7053b0633fcd61bcc


In [11]:
print(db.collection_names(include_system_collections=False))
print(db_new.collection_names(include_system_collections=False))

['training-collection']
['collection-new']


In [12]:
# QUERIES in MOngo DB

import pprint

# find_one

print("**** Get the first document in the collection **** \n")
pprint.pprint(collection.find_one())

print()

print("**** Get the document based on a condition in the collection **** \n")
pprint.pprint(collection.find_one({'author':'Mike1'}))

print("\n**** If no result of find_one returns None **** \n")
pprint.pprint(collection.find_one({'author':'Mike2'}))

**** Get the first document in the collection **** 

{'_id': ObjectId('5bbd9ff7053b0633fcd61bcb'),
 'author': 'Mike1',
 'date': datetime.datetime(2018, 10, 10, 6, 45, 11, 334000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}

**** Get the document based on a condition in the collection **** 

{'_id': ObjectId('5bbd9ff7053b0633fcd61bcb'),
 'author': 'Mike1',
 'date': datetime.datetime(2018, 10, 10, 6, 45, 11, 334000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}

**** If no result of find_one returns None **** 

None


In [15]:
# QUERIES using ObjectId

id_ret = collection.insert_one({'test':'test1', 'test2':'test2'}).inserted_id
print(id_ret)

<class 'bson.objectid.ObjectId'>


In [14]:
find_res = collection.find_one({'_id':id_ret})
print(find_res)

{'_id': ObjectId('5bbda11d053b0633fcd61bcd'), 'test': 'test1', 'test2': 'test2'}


In [16]:
# ObjectId is not a string , it a differnt data type

id_ret_str = str(id_ret)
collection.find_one({"_id": id_ret_str}) # No result

In [ ]:
# USE CASE

# A common task in web applications is to get an ObjectId from the request URL and find the matching document. 
# In this case convert the ObjectId from a string before passing it to find_one as below.

from bson.objectid import ObjectId

# The web framework gets post_id from the URL and passes it as a string
def get(id_ret_str):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(id_ret_str)})
    print(document)

In [17]:
# BULK INSERT

# insert_many
# returns a list of object id's

new_posts = [{"author": "test_data",
               "text": "Another post!",
               "tags": ["bulk", "insert"],
               "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
               "title": "MongoDB is fun",
               "text": "and pretty easy too!",
               "date": datetime.datetime(2009, 11, 10, 10, 45)}]

result = collection.insert_many(new_posts)
result.inserted_ids

[ObjectId('5bbda204053b0633fcd61bcf'), ObjectId('5bbda204053b0633fcd61bd0')]

In [18]:
# QUERYING more than one document

# find()
# returns a cursor using which we can interate through all the documents
import pprint
cursor = collection.find({'author':'Mike1'})

for entry in cursor:
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    pprint.pprint(entry)
    print("\n",entry['author'],"\n")



%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
{'_id': ObjectId('5bbd9ff7053b0633fcd61bcb'),
 'author': 'Mike1',
 'date': datetime.datetime(2018, 10, 10, 6, 45, 11, 334000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}

 Mike1 



In [19]:
#Count the number of rows in a collection

num_row = collection.count()
print(num_row)

collection.find({"author": "Mike"}).count()

5


0

In [22]:
# Update document 

from bson.objectid import ObjectId

print (id_ret)
try:
    
    collection.update_one(
                {"_id": id_ret},
                {
                    "$set": {
                        "test":"Ritu"
                    }
                }
            )
except Exception as e:
    print(str(e))

5bbda155053b0633fcd61bce


In [ ]:
# Delete Document 

# delete_one() or delete_many()

result = collection.delete_many({"author": "Mike1"})

# To see the number of documents deleted :
print(result.deleted_count)

# Remove all documents using delete_many()
result= collection.delete_many({})

# Delete all documents using collection.remove()
result = collection.remove() 

# To Drop a Collection
db.my_collection.drop()